# 2022-47 Base Year Forecast Output QC
Test Plan: https://sandag.sharepoint.com/qaqc/_layouts/15/Doc.aspx?sourcedoc={f8b3d630-1290-445b-99a1-2fa9041ade92}&action=edit

Documentation: https://sandag.sharepoint.com/:w:/r/qaqc/_layouts/15/Doc.aspx?sourcedoc=%7B3AF20D75-0A22-4B9C-9CC4-85B3EEC294E6%7D&file=MGRABased_input_ABM_2019_process_notes.docx 

### Library Imports

In [1]:
import pandas as pd
import pyodbc

from pathlib import Path

# ignore warning relating to pandas and pyodbc (just ignore all warnings)
import warnings
warnings.filterwarnings("ignore")

### Download Data


In [2]:
def download_csv_data(user):
    """
    This function downloads csv data for the 2019 Forecast Output

    :param user:    The user trying to download the data. Mostly here so that others can more 
                    easily run my code

    :returns:       Tuple with (mgra data, region data)
    """

    # Data is stored in this folder
    data_folder_path = Path(f"C:/Users/{user}/San Diego Association of Governments/" \
        "SANDAG QA QC - Documents/Projects/2022/2022-47 Base Year Forecast Output QC/Data/")

    # Define the files we need from SharePoint
    sp_files = ["mgra_ind.csv", "region_ind.csv"]

    # Download the data and put them into dfs
    mgra_ind = pd.read_csv(data_folder_path / sp_files[0])
    region_ind = pd.read_csv(data_folder_path / sp_files[1])

    # Return the data in tuple format
    return mgra_ind, region_ind

In [3]:
# Get the csv data
mgra, region = download_csv_data("eli")

In [37]:
def download_SQL_data():
    """
    This function downloads SQL data for the 2019 Forecast Output

    :returns:       Tuple with (mgra data, region data)
    """
    # Create the SQL connection to download juristiction data
    connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
        'Server=DDAMWSQL16.sandag.org;'
        'Database=dpoe_stage;'
        'Trusted_Connection=yes;')

    # Get only the data we want (Series 14; mgra_id to jurisdiction table)
    query = """SELECT mgra_id, jurisdiction FROM demographic_warehouse.dim.mgra_denormalize
        WHERE series=14"""
    jurisdiction = pd.read_sql_query(query, connection)

    # Get the data for every single fact table
    fact_table_names = ["age", "age_sex_ethnicity", "ethnicity", "household_income", "housing", "jobs",
        "land_use", "population", "sex"]
    fact_data = {}
    for table_name in fact_table_names:
        fact_data[table_name] = pd.read_sql_query(f"""
            SELECT * FROM demographic_warehouse.fact.{table_name} 
            WHERE datasource_id=44
            AND yr_id=2019""", connection)

    # For each fact table, add on the jurisdiction information using the mgra_id to jurisdiction 
    # table
    for table_name in fact_table_names:
        fact_data[table_name] = fact_data[table_name].merge(
            jurisdiction, left_on="mgra_id", right_on="mgra_id")

    return fact_data

fact_tables = download_SQL_data()

### Tests

#### Tests for Population Variables

In [4]:
# pop = total population
# hhp = total household population (exlucde gq pop)
# gq_total = total gq population, derived from sum of population in civialn gq and military gq
population_variables = ["pop", "hhp", "gq_total"]

# Create the gq_total column in both dfs
mgra["gq_total"] = mgra["gq_civ"] + mgra["Group Quarters - Military (gq_mil)"]
region["gq_total"] = region["gq_civ"] + region["Group Quarters - Military (gq_mil)"]

In [5]:
# Check the population variables match up between mgra and region
mgra[population_variables].sum() == region[population_variables]

,pop,hhp,gq_total
0,True,True,True


In [62]:
# Check the population variables match up between Jurisdiction and Region
population_checks = {}

# Check total population 
population_checks["pop"] = (fact_tables["population"]["population"].sum() == region["pop"])

# Check total household population
non_gq_table = fact_tables["population"][fact_tables["population"]["housing_type_id"] == 1]
population_checks["hhp"] = (non_gq_table["population"].sum() == region["hhp"])

# Check total gq population
gq_table = fact_tables["population"][fact_tables["population"]["housing_type_id"] != 1]
population_checks["gq_total"] = (gq_table["population"].sum() == region["gq_total"])

pd.DataFrame.from_dict(population_checks)

,pop,hhp,gq_total
0,True,True,True


#### Tests for Housing Variables

In [ ]:
housing_variables = ["hs", "hh", "hhs"]

#### Tests for Employment Variables

#### Tests for School Enrollment

#### Tests for College Enrollment

#### Tests for Income

#### Tests for Ethnicity by Category

#### Tests for Age

#### Tests for Total Hotel Rooms